In [3]:
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [4]:
import pycrfsuite

from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report

def readCorpus(fpath):
    sents = []
    with open(fpath) as fd:
        sent = []
        for l in fd:
            #lt = l.strip().decode("utf8")
            lt = l.strip()
            if not lt:
                sents.append(sent)
                sent = []
            else:
                w_t = lt.split('\t')
                sent.append([w_t[0], w_t[1]])
    return sents

def word2features(sent,i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        #'bias',
        'word.lower=' + word.lower(),
        #'word[-3:]=' + word[-3:],
        #'word[-2:]=' + word[-2:],
        #'word.isupper=%s' % word.isupper(),
        #'word.istitle=%s' % word.istitle(),
        #'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        #'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        pass
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            #'-1:word.istitle=%s' % word1.istitle(),
            #'-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            #'-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        pass
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            #'+1:word.istitle=%s' % word1.istitle(),
            #'+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            #'+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, label in sent]
def sent2tokens(sent):
    return [token for token, label in sent]

In [6]:
train_sents = readCorpus("CrfTrain.TXT")
print(train_sents[0])

[['same', 'O'], ['as', 'O'], ['picture', 'O'], ['but', 'O'], ['string', 'O'], ['chahi', 'O'], ['ali', 'O'], ['moto', 'O'], ['cha', 'O'], ['but', 'O'], ['nice', 'B-ENG'], ['quality', 'O']]


In [8]:
sent2features(train_sents[0])[0]

['word.lower=same', 'postag=O', 'BOS', '+1:word.lower=as', '+1:postag=O']

In [9]:
x_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

In [14]:
trainer = pycrfsuite.Trainer(algorithm='l2sgd',verbose=False)
for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)
print("append train set done.")
trainer.set_params({
    #'c1': 0.01,
    'c2': 0.01,
    'max_iterations': 100,
    'feature.possible_transitions': True
    })

trainer.train("trained_model")
test_object = pycrfsuite.Tagger()
test_object.open("trained_model")

append train set done.
